TensorFlowを始めよう　メモ

TensorFlowの実行環境を整え、基本知識についておさらいしましょう。
Kerasのサイト https://keras.io/ja/

TensorFlowはKerasを土台にして開発されたライブラリです。すなわち、TensorFlowを知るとはKerasを知ることに繋がり、Kerasを知る事はニューラルネットワークを知る事に繋がります。もちろんニューラルネットワークを理解するためには数学を知っている必要があるのです。

１　ソースコードの解読

まず、TensorFlow.org のチュートリアルから以下のソースコードを眺めてみます

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)


このソースコードを読み解くにあたり、１行１行詳しく調べて全体を理解しようと試みます。
１ー１　データの入力・変換
＜２行目の解釈＞
mnist = tf.keras.datasets.mnist
コードの２行目からmnistという知らない英単語が出てきました。概要を理解するため以下のページを参照してみます。
https://en.wikipedia.org/w/index.php?title=MNIST_database

ざっくりまとめると、ディープラーニングをするためのデータだそうです。
MNIST データベース　ー＞　ディープラーニング　ー＞　結果出力
上記の流れになると思います。
本来、ディープラーニングをするためのデータは統計的手法を用いて回収する必要があります。しかし、今回のケースではMNISTデータベースが元からあるそうです。


＜４行目の解釈＞
(x_train, y_train),(x_test, y_test) = mnist.load_data()
さて、４行目からタプル形式でMNISTデータを読み込みます。
＜５行目の解釈＞
x_train, x_test = x_train / 255.0, x_test / 255.0
ここで、なぜデータを255.0で割っているのか気になります。そこで、調べてみることにしました。適当にインターネットを検索してすぐにたどり着いた結果が以下のURLです。
http://may46onez.hatenablog.com/entry/2016/07/14/122047

そうです。CG技術者にとっては当たり前の知識ですが画素数が２５５の場合は255.0で割るのです（画素は０〜２５５で表されます）。なぜか？Kerasは０から１の範囲の要素を持つベクトルデータを処理するからだ。

まとめ：２行目から４行目まではデータの入力と変換です。おそらく、ここからモデルを導入し、レイヤーを多重化して深層学習（ディープラーニング）を実現していきます。本腰を入れます。

１−２　モデルの構築
＜７行目の解釈＞
model = tf.keras.models.Sequential
モデルを定義し、レイヤーを代入していきます。Sequentialは連続の意味で、一般的に最もよく使われる深層学習のモデルであるとTensorFlow.orgは述べています。
＜８行目の解釈＞
  tf.keras.layers.Flatten(),
さて、このFlatten( )関数は何をしているのか？興味津々でインターネットで調べてみると直ちに以下のサイトにたどり着いた。

http://aidiary.hatenablog.com/entry/20161120/1479640534

なるほど、画像データは四次元のテンソルデータであり、一次元のデータに変換する必要があるのだ。このFlatten( )関数の作用がわかった。
＜９行目の解釈＞
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
いよいよ、深層学習の真髄を見出せそうだ。まず、Denseとは何か？
しばらく考えてインターネットで探した結果、以下のURLを見つけた。
https://teratail.com/questions/105940
なるほど密度か！と思った瞬間に５１２の意味も理解する。画素は２５６種類（０〜２５５）であるため、あるノードにインプットするエッジは２５６、アウトプットするエッジも２５６であるから合計５１２ではないか！（まだ怪しい解釈だ）
ちなみにactivation=tf.nn.relu のreluは活性化関数です。
https://ja.wikipedia.org/wiki/活性化関数
＜１０行目の解釈＞
tf.keras.layers.Dropout(0.2),

Dropout関数は一体何をしているのでしょうか？適当にインターネットから以下のURLを見つけました。
http://arakan-pgm-ai.hatenablog.com/entry/2017/09/09/003000
過学習を抑制するためにあるそうです？
＜１１行目の解釈＞
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
どうやらソフトマックス回帰を実施しているそうです。
https://blog.amedama.jp/entry/2016/02/08/204551

７行目から１１行目で深層学習のためのモデルの構造が作られています。そのモデル（抽象的なもの）を具現化するために次のステップが必要になります。
１−３　モデルのコンパイル
＜１３行目の解釈＞
model.compile(optimizer='adam',
１３行目からのコンパイル(compile)メソッドでモデルをコンパイルします。
http://bluewidz.blogspot.com/2017/09/keras.html
上記のURLからオプティマイザーの種類がわかります。
ざっくりオプティマイザーをまとめると、ディープラーニングのトレーニングの種類のことです。
＜１４行目の解釈＞
              loss=‘sparse_categorical_crossentropy',
Loss Functionは日本語で目的関数と呼ばれます。
https://en.wikipedia.org/wiki/Loss_function
この関数は仮設関数を導くための関数と以下のURLは説明しています。
http://tech.mof-mof.co.jp/blog/machine-learning-cost-function.html
＜１５行目の解釈＞
              metrics=['accuracy'])
よくわからないので以下のURLを見つけました。
https://qiita.com/mytk0u0/items/d11ce0c7393e96149fa6
簡単に言うと正答率を評価するそうです。しかし、特殊な方法で正答率を導きます。

以上の１３行目から１５行目は終わりです。
今までの流れは、（１）データの入力　（２）モデルの構成　（３）モデルのコンパイル　です。最後にディープラーニングを実行します。

１−４　ディープラーニングの実行

＜１７行目の解釈＞
model.fit(x_train, y_train, epochs=5)
この関数は以下のURLによると、機械学習の回数を指定するそうです。
https://keras.io/ja/models/sequential/
＜１８行目の解釈＞
model.evaluate(x_test, y_test)

https://keras.io/ja/models/sequential/
上記のURLをもう一度参照すると、損失数を出力するそうです。

２　実行結果
いよいよ、実行してみます。出力結果は以下の通りです。

SFV-Server:git_test hikaru$ python kerastest.py
dyld: warning, LC_RPATH $ORIGIN/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal.so___Utensorflow in /Library/Python/2.7/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so being ignored in restricted program because it is a relative path
Downloading data from https://s3.amazonaws.com/img-datasets/mnist.npz
11493376/11490434 [==============================] - 13s 1us/step
11501568/11490434 [==============================] - 13s 1us/step
Epoch 1/5
2018-08-16 15:04:29.225522: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
60000/60000 [==============================] - 21s 346us/step - loss: 0.2221 - acc: 0.9334
Epoch 2/5
60000/60000 [==============================] - 18s 294us/step - loss: 0.0975 - acc: 0.9698
Epoch 3/5
60000/60000 [==============================] - 17s 284us/step - loss: 0.0709 - acc: 0.9779
Epoch 4/5
60000/60000 [==============================] - 17s 282us/step - loss: 0.0539 - acc: 0.9829
Epoch 5/5
60000/60000 [==============================] - 17s 291us/step - loss: 0.0433 - acc: 0.9866
10000/10000 [==============================] - 1s 95us/step

３　ディスカッション
　ディープラーニングの実行のために、（１）開発環境の構築　（２）ディープラーニングの知識　（３）プログラミングの技術　（４）データ収集　などが必要である。
さらに深く理解するために数学の知識を深め、Kerasのサイトで実行方法の文法を暗記していく必要があるだろう。TensorFlowの本でも買おうかと考える。

これからの学習の流れとして、
（１）Kerasの学習
（２）TensorFlowの学習
（３）自己流で現実世界に適用
とします。
（（１）と（２）は同時並行でいい気がする）

今回の学習で何を知らないのか、何を学習するべきかがわかった。